(onboardone)=
# Import a World Bank Eviews model from its solution file (.wf1)# Import a World Bank Eviews model from its solution file (.wf1)
This notebook takes a .wf1 workfile and transforms it into a modelflow model.

Most standard World Bank models should work with limited intervention.  Some using unusual techniques or variable definitions may require intervention by the user.

## Overview of the import process

The overall import process is performed by a special a ModelFlow class named:**GrabWfModel**. Certain steps require the use of EViews itself.  This howto was designed using EViews version 12, but has been tested with versions 13 and 14.

Steps to follow:
 1. Start EViews and open the solution file 
    -    If returning to this step after an initial error perform transformations on the data (needed in some cases where special EViews functions are used in the model).
    -    The UNlink the model. This transforms linked equations into explicit equations in the EViews model object. 
    -    Save the revised model as a .wf2 file  (the .wf2 is a JSON format that can be easily read into python), with the same name as the original file but with "\_modelflow" appended to the name.
    
 5. Close Eviews.
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSAOptions strinng from the views file is  extracted, to be saved in the ModelFlow pcim file. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
     - Some special series are generated, for the case where an inline function is used in EViews that can not be translated directly into modelflow model specifications
     - The model for fitted values is simulated in the specified timespan
     - The model for add_factors is simulated in the timespan set in MFMSAOptions
 10. The data descriptions are extracted into a dictionary. 
    - Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  
  


## Prerequisites  

The import process requires EViews and tehrefore cannot be run on a non-windows computerthat does not have EViews 12 or later installed.  It also assumes that the python library `pyeviews` has been installed (`conda install pyeviews`).  `pyeviews` uses the EViews com interface to control eviews, allowing a python script to directly manipulate parts of the eviews model using eviews itself.  This is required only for the import process.


In [1]:
#tell matplotlib we are running in a notebook
%matplotlib Notebook 

In [2]:
from pathlib import Path

from modelclass import model 
from modelgrabwf2 import GrabWfModel  #this is the class (part of the Modelflow Library) that will translate
                                      # the ewbvis model and equations into modelflow business logic and
                                      # import the data into pandas
model.widescreen() # Makes modelflow aware we may be running under a wider than normal screem
model.scroll_off() # tells modelflow not to scroll outputs

<IPython.core.display.Javascript object>

In [3]:
 %load_ext autoreload
 %autoreload 2

## Model specific transformations 

Some expressions found in World Bank equations are functions that have no direct equivalent in python. `GrabWfModel()` is aware of many of these and performs he necessary adjustments automatically. Others it may not be aware of.  

In cases where manual adjustments are necessary, the `GrabWfModel()`procedure will error out and the equations will have to be re-written perhaps as two or more equations manually. 

We perform these changes in EViews using the `pyeviews` function by entering EViews command and submitting them to the `GrabWfModel()`function. 

For example, and EViews equation may include a cross equation constraint by specifying a parameter in one equation to be equal to the value of a parameter in another equation. 

In an EViews model this might be written as 

`equation _mycrossequation.ls dlog(vary) = c(1) + SomeOtherEquation.coeff(2)*Somevar`

which would instruct EViews to run a least squared regression estimating only the intercept `c(1)` and imposting as the slope variable the second coefficient from the second equation `SomeOtherEquation.coeff(2)`.  Modelflow does not have this dynamic mechanism to refer to coefficients in other equations so we must rewrite the equation by first assigning the value of Secondequation.coeff(2) to a scalar and the using that scalar in the equation instead.

We would do this by executing (in EViews) the following  line.

```
'Eviews code that assigns to a scalar the value of the coeeficint
Scalar _SomeOtherEquation_at_COEF_2 = SomeOtherEquation.@COEF(+2)
```
We will then want to rewrite the equation (in modelflow) substituting the expression 
`SomeOtherEquation.@coeff(2)` with the `scalar_SomeOtherEquation_at_COEF_2`.

This is accomplished by passing to `GrabWFModel()` two parameters, the first that contains the EViews code to be run and the second the change to the equation we need.  These are then run by `GrabWFModel()` before generating the Modelflow version of the model.

Below is a real-life example required when importing a model for Moldova.

```
mda_eviews_run_lines = ['Scalar _MDASBBREV_at_COEF_2 = _MDASBBREV.@COEF(+2)']
mda_trans = lambda input: input.replace('_MDAsbbrev.@coef(2)','_MDASBBREV_at_COEF_2')  
```

In this second example, the original EViews equation included an expression that calculated the mean value of a variable on the fly and used it in an equation.  Here we use two EViews commands `smpl @all` and `series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")` to create a variable that contains the value the expression would generate and inserts that into the equation instead of the expression.

```
# Another example 
ago_eviews_run_lines = ['smpl @ALL','series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")']
ago_trans = lambda  input : input.replace('@MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")','MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD') 
```

:::{admonition} Note
Multiple lines of code can be executed by setting the eviews_runlines variable as a list as in this instance.
:::

|The GrabWBModel parameters| Purpose|
|:--|:--|
|**eviews_run_lines**|Specifies a comma-delimited list of Eviews command lines to be run. Typically these generate a variable to stand in for an inline function that appears in an equation in the model object. There is not limit to the number of EViews commands that can be submitted| 
|**country_trans**|Specifies a comma-delimited list of replacements (python code) to be done to the formulas before they are processed further. |


Example use:
```
all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                  eviews_run_lines= mda_eviews_run_lines,
                  country_trans    =  mda_trans,
                  make_fitted = True,        # make equations for fitted values of stochastic equations 
                  do_add_factor_calc=True,   # Calculate the add factors which makes the stochastic equations match                     
                  fit_start = 2000,          # Solve start for the model
                  fit_end   = None,          # Solve End date, 
                                             # if None taken it will use the value specified 
                                             # in the EViews MFMSAOptions variable 
                    disable_progress =True   # Supresses the progress bar
                           ) 
```

Not all inline EViews functions (@ functions) will throw errors.  The import routine knows how to deal with many of them.  The table below outlines those it is aware of and what they do.

|EViews in line function | What it does |
|:--|:--|
|@During(Date,[Date])|Generates a dummy with value one at date (or over a range of dates if more than one date is specified|
|@Recode(variable,Logical condition)|Generates a dummy with value one for those dates where the logical condition is true|




## An example of a clean import

Below the Iraq model is imported.  The eviews_run_lines and couontry+trams parameters are omitted to see if the model canbe imported directly with the `GrabWfModel()` routine.


In [4]:
try:
    all_about_irq = GrabWfModel(r'wfs/irqsoln.wf1', 
                  #eviews_run_lines= irq_eviews_run_lines,  #
                  #country_trans    =  irq_trans,
                    make_fitted = True,        # make equatios for fitted values of stocastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True     # Better for jupyter book 
                           ) 
except  Exception as e:
    print(f'Error converting model: {e}')


Reading c:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\irqsoln.wf1
Assummed model name: IRQ
The model: IRQ is unlinked 
Writing C:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\irqsoln_modelflow.wf2
Model name: IRQ

Processing the model:IRQ
Check for Eviews @ which are not caught in the translation
Default WB var_group loaded
Variable description in wf1 file read
Default WB var_description loaded self.cty='' len(var_description)=37
var_description loaded from WF len(this)=115
testmodel calculated  
Calculation of add factors for IRQ calculated  


This import worked smoothly and returns a structure all_about_irq that contains the `ModelFlow` model object, some initial pandas and other components.  These can be referenced later.

**Some properties of the all_about data structure**
|Property Name| Contents |
|:--|:--|
|all_frml|The formulas of the imported model|
|all_frml_dict| A dictionary containing all the formulas of the model|
|base_input| A dataframe of the data that was used to initialize th model|
|cty|The three-letter ISO mnemonic for the country|
|country_name|The English name of the country|
|eviews_run_lines|List of Eviews commands run in the pre-processor phase of importing.|
|filename|Name of the originally imported EViews file|
|fit_end|End period over which the model is solved|
|fit_start|Start of period over which the model is solved|
|freq|Frequenecy of the  model A=Annual; Q=Quarterly; M=Monthly|
|mfmsa_options|Options variable from the EViews file|
|mfmsa_quasiIdentities|List of Eviews quasi identities|
|mfmsa_start_end|Eviews Start and End Solve periods|
|modelname|Name of model|
|var_description|Dictionary of variable descriptions|
|var_groups|EViews groups defined|
|wb_default_descriptions|Default variable descriptions (Glossary) in WB models|
|missing_descriptions|Dictionary of variables with missing descriptions|

country_df_trans
 'country_trans',
  'dfmodel',
 'disable_progress',
 'do_add_factor_calc',
 'end',
 
 'mmodel',
 'model_all_about',
 'model_description',
 
 




## Check if the imported equations returns their  historical values

An essential feature of a model is that if nothing is changed it returns the same results.  For the imported model, therefore when it is solved it should (within tolerances for rounding errors) return the same data that it was initiated with it.

The test_model routine solves each equation and checks that the add factors are the same as those input and returns those where the difference exceeds a given tolerance.

Below the absolute difference was larger than 1 for 7 variables but this was less than e-5 percent or e-7 (if we don't multiply by 100), which is a very small deviation. The amount of deviation that the solver will tolerate can be set as an option and e-6 is a commonly used level.

We judge the import to be successful


In [5]:
all_about_irq.test_model(all_about_irq.start,all_about_irq.end,maxerr=100,tol=1,showall=0)   # tol determins the max acceptable absolute difference 

IRQ calculated  

Chekking residuals for IRQ 2017 to 2040

Variable with residuals above threshold
IRQNYGDPFCSTKN              , Max difference:     2.48473889 Max Pct    0.0000012537% It is number    39 in the solveorder and error number 1
IRQNYGDPNOILKN              , Max difference:     1.62679964 Max Pct    0.0000017770% It is number    46 in the solveorder and error number 2
IRQNECONGOVTKN              , Max difference:     2.65173910 Max Pct    0.0000063979% It is number    50 in the solveorder and error number 3
IRQNEGDETTOTKN              , Max difference:     2.20873368 Max Pct    0.0000014688% It is number    66 in the solveorder and error number 4
IRQNECONPRVTCN              , Max difference:     2.24584737 Max Pct    0.0000013446% It is number    67 in the solveorder and error number 5
IRQNYGDPMKTPCN              , Max difference:     4.49169481 Max Pct    0.0000012072% It is number    79 in the solveorder and error number 6
IRQNYGDPMKTPKN              , Max difference:    

## Extract the model and the baseline

Here we extract from all_about various components that we will use below.


In [6]:
mirq    = all_about_irq.mmodel       # the model instance  
baseline = all_about_irq.base_input  # the dataframe used to initialize the model

## Run the model 

The below solves the model with no change in the exogenous variables to check that it returns the same input data. 
 

In [28]:
res = mirq(all_about_irq.base_input,2020,2040,silent=1,alfa=1,ldumpvar=0)
mirq.basedf = all_about_irq.base_input

## And the simulation result is also fine. 
Here the percent difference is displayed

In [29]:
mirq['irqGGEXPCAPTCN irqNYGDPMKTPCN irqGGDBTTOTLCN irqBNCABFUNDCD']

## Look at a stochastic variable 

The all_frml_dict contains a listing of all the equations and the different forms their equations take, notably the original equation, The EViews versions (which should be the same as the original), the pre-processed version (after any EViews inline functions have been replaced), the normalized version -- the EViews equation re-written to solve for the level of the dependent variable.

The first variable which is the GDP identity is relatively straightforward to read, the second which is the equation for the consumer price deflator is more complex because of the estimated coefficients (c1), C(2) in the eviews form and with the estimated value of those coefficients substituted in for the normalized version.

In [10]:
all_about_irq.all_frml_dict['IRQNYGDPMKTPKN'].fprint


Endo_var        : IRQNYGDPMKTPKN
Original        : IRQNYGDPMKTPKN=IRQNECONPRVTKN+IRQNECONGOVTKN+IRQNEGDIFTOTKN+IRQNEGDISTKBKN+IRQNEEXPGNFSKN-IRQNEIMPGNFSKN+IRQNYGDPDISCKN
Preprocessed    : IRQNYGDPMKTPKN=IRQNECONPRVTKN+IRQNECONGOVTKN+IRQNEGDIFTOTKN+IRQNEGDISTKBKN+IRQNEEXPGNFSKN-IRQNEIMPGNFSKN+IRQNYGDPDISCKN
Normalized      : IRQNYGDPMKTPKN = IRQNECONPRVTKN+IRQNECONGOVTKN+IRQNEGDIFTOTKN+IRQNEGDISTKBKN+IRQNEEXPGNFSKN-IRQNEIMPGNFSKN+IRQNYGDPDISCKN
Eviews          : @IDENTITY IRQNYGDPMKTPKN  = IRQNECONPRVTKN  + IRQNECONGOVTKN  + IRQNEGDIFTOTKN  + IRQNEGDISTKBKN  + IRQNEEXPGNFSKN  - IRQNEIMPGNFSKN  + IRQNYGDPDISCKN


In [11]:
all_about_irq.all_frml_dict['IRQNECONPRVTXN'].fprint


Endo_var        : IRQNECONPRVTXN
Original        : DLOG(IRQNECONPRVTXN)=-0.2*(LOG(IRQNECONPRVTXN(-1))-0.2*LOG(IRQNYGDPFCSTXN(-1))-(1-0.2)*LOG(IRQNEIMPGNFSXN(-1))-LOG((1+IRQGGREVGNFSER(-1)/100))-LOG(ABS(1.12510183905574)))+0.1*DLOG(IRQNYGDPFCSTXN)+(1-0.1)*DLOG(IRQNEIMPGNFSXN)+DLOG(1+IRQGGREVGNFSER/100)+0.1*IRQNYGDPGAP_/100
Preprocessed    : ((LOG(IRQNECONPRVTXN))-(LOG(IRQNECONPRVTXN(-1))))=-0.2*(LOG(IRQNECONPRVTXN(-1))-0.2*LOG(IRQNYGDPFCSTXN(-1))-(1-0.2)*LOG(IRQNEIMPGNFSXN(-1))-LOG((1+IRQGGREVGNFSER(-1)/100))-LOG(ABS(1.12510183905574)))+0.1*((LOG(IRQNYGDPFCSTXN))-(LOG(IRQNYGDPFCSTXN(-1))))+(1-0.1)*((LOG(IRQNEIMPGNFSXN))-(LOG(IRQNEIMPGNFSXN(-1))))+((LOG(1+IRQGGREVGNFSER/100))-(LOG(1+IRQGGREVGNFSER(-1)/100)))+0.1*IRQNYGDPGAP_/100
Normalized      : IRQNECONPRVTXN = (IRQNECONPRVTXN(-1)*EXP(IRQNECONPRVTXN_A+ (-0.2*(LOG(IRQNECONPRVTXN(-1))-0.2*LOG(IRQNYGDPFCSTXN(-1))-(1-0.2)*LOG(IRQNEIMPGNFSXN(-1))-LOG((1+IRQGGREVGNFSER(-1)/100))-LOG(ABS(1.12510183905574)))+0.1*((LOG(IRQNYGDPFCSTXN))-(LOG(IR

## Look a all the modelflow frmls
Notice after the "original" model the equations for the "fitted" values have been added. <br>
Also in the end of the listing the specification of the model which calculates the add factors if a variable is fixed. When processing the equations the ```model``` class will process this this model separately and create a model instance 
which is used to calculate add factors in case 

In [13]:
print(mirq.equations)

FRML <IDENT> IRQNYGDPPOTLKN = IRQNYGDPTFP*(IRQLMEMPSTRL**IRQNYYWBTOTLCN_)*(IRQNEGDIKSTKKN(-1)**(1-IRQNYYWBTOTLCN_)) $
FRML <IDENT> IRQNYGDPPOTWKN = IRQNYGDPPOTLKN+IRQNVOILTOTLKN $
FRML <IDENT> IRQNYGDPGAP_ = ((IRQNYGDPNOILKN/IRQNYGDPPOTLKN-1)*100) $
FRML <IDENT> IRQNYGDPGAPW_ = ((IRQNYGDPMKTPKN+IRQNYTAXNINDKN)/IRQNYGDPPOTWKN-1)*100 $
FRML <IDENT> IRQNEGDIKSTKKN = IRQNEGDIKSTKKN(-1)*(1-IRQDEPR)+IRQNEGDIFTOTKN $
FRML <IDENT> IRQNEKRTTOTLXN = IRQNEGDIFTOTXN*(IRQNEKRTPREMFR/100+IRQDEPR+IRQFMLBLPOLYFR/100-IRQINFLEXPT/100) $
FRML <IDENT> IRQPSTAR = (IRQNEWRTTOTLXN**IRQNYYWBTOTLCN_*IRQNEKRTTOTLXN**(1-IRQNYYWBTOTLCN_))/(IRQNYGDPTFP*IRQNYYWBTOTLCN_**IRQNYYWBTOTLCN_*(1-IRQNYYWBTOTLCN_)**(1-IRQNYYWBTOTLCN_)) $
FRML <IDENT> IRQLMLBFTOTL = IRQLMPRTTOTL_/100*IRQSPPOPWORK $
FRML <IDENT> IRQNYYWBTOTLCN = IRQNEWRTTOTLXN*IRQLMEMPTOTL $
FRML <DAMP,STOC> IRQNEWRTTOTLXN = (IRQNEWRTTOTLXN(-1)*EXP(IRQNEWRTTOTLXN_A+ (-0.572328530761233*(LOG(IRQNEWRTTOTLXN(-1))-LOG(IRQNECONPRVTXN(-1)*IRQNYGDPPOTLKN(-1)/IRQLMEM

In [ ]:
with mirq.set_smpl(2020,2023):
    print(mirq.IRQGGEXPCAPTCN.show)                        

## Save the successfully imported file

In [14]:
mirq.modeldump('irq.pcim')

In [15]:
!dir *.pcim

 Volume in drive C has no label.
 Volume Serial Number is C2DB-095E

 Directory of c:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews

13-12-2024  08:50           220.807 bol.pcim
13-12-2024  08:55           238.021 irq.pcim
22-11-2024  23:44           220.847 test.pcim
               3 File(s)        679.675 bytes
               0 Dir(s)  649.838.759.936 bytes free


## An example where the initial import fails because of a specifity in the orginal EViews model

The below command tries to import a model for Iraq without any modification.  The import fails, with `GrabWfModel()` reporting:

```
Processing the model:BOL
Probably errors as @ in lines:

Eviews line      :@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3
Original line     :@IDENTITY BOLBFCAF2BOP_  = (1  - DURING_1980_2021)  * @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)  * RESERVERATIO3
New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3
```


Below we try to onboard the model for Iraq directly without any intervention.

In [19]:

BOL_Eviews=['smpl @ALL',"series _Reserveratio3_at_Between= @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)"];
BOL_trans=lambda input: input.replace("@BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)","__RESERVERATIO3_AT_BETWEEN")
try:
    all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                      #eviews_run_lines= BOL_Eviews,
                      #country_trans    =  BOL_trans,
                        make_fitted = True,        # make equations for fitted values of stochastic equations 
                        do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                        fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                        fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                        disable_progress =True     # Better for jupyter book 
                               ) 
except Exception as e:
    print('*** ERROR ***','\n',e)
    


Reading c:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln.wf1
Assummed model name: BOL
The model: BOL is unlinked 
Writing C:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln_modelflow.wf2
Model name: BOL

Processing the model:BOL
Check for Eviews @ which are not caught in the translation
Probably errors as @ in lines:

Eviews line      :@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3
Original line     :@IDENTITY BOLBFCAF2BOP_  = (1  - DURING_1980_2021)  * @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)  * RESERVERATIO3
New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3
*** ERROR *** 
 @ in lines 


## Debugging the error

Many EViews models can be converted to ModelFlow. However, sometimes plain vanilla conversion doesn't work. In broad terms, experience shows three types of issues:  

1. **Unknown EViews @functions in equations.**
  The `GrabWfModel` function knows the most common Eviews @functions, however not all. 
2. **Syntax errors**. Conversion and  normalization of EViews equations to python statements can result in syntax errors.   
3. **The ModelFlow model don't calculate the same values as in the `.wf1` file.**. The `GrabWfModel` function calculates adjustment factors for all stochastic equations so that the equations yield the same values as in the `.wf1` file. However, this is not always the case, particularly if the value of an identity has been explicitly set. It can also be caused by limited numerical prescision.  


GrabWfModel knows some Eviews @<functions> but not all. As we can see from the line: 

 >New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3

 GrabWfModel can not make @Between go away. That is causing trouble.

The EViews help tells us:

```
@between(series, val1, val2)  Creates a dummy variable equal to 1 for observations where series is greater than or equal to val1 and less than or equal to val2.
```

In python @between(series, val1, val2) is equivalent to: 
    
 >  (1. * float(val1 <= series <= val2)))  
    
We can reproduce the logic by adding a python commmand to perform the substitution of @between in the equation with a equivalent 
python expression 


```BOL_trans = lambda text: re.sub(
    r"@BETWEEN\(\s*(\w+)\s*,\s*([^,]+)\s*,\s*([^)]+)\s*\)",
    r"(1.0*float(\2 <= \1 <= \3))",
    text
)```


In [24]:
# To show how the substitution works
import re
BOL_trans = lambda text: re.sub(
    r"@between\(\s*(\w+)\s*,\s*([^,]+)\s*,\s*([^)]+)\s*\)",
    r"(1.0*float(\2 <= \1 <= \3))",
    text
)


# Usage example
text = '@between(3  , 1  / 2  , 100000)'
result = BOL_trans(text)

print(result)


(1.0*float(1  / 2   <= 3 <= 100000))


In [25]:
# some results 
print(f'{1.0*((1  / 2)   <= 30 <= 100000)=}')
print(f'{1.0*((1  / 2)   <= 30000 <= 100000)=}')
print(f'{1.0*((1  / 2)   <= 0 <= 100000)=}')

1.0*((1  / 2)   <= 30 <= 100000)=1.0
1.0*((1  / 2)   <= 30000 <= 100000)=1.0
1.0*((1  / 2)   <= 0 <= 100000)=0.0


In [27]:
# Now the substitution is implementet 
import re
BOL_trans=lambda text: re.sub(
    r"@between\(\s*(\w+)\s*,\s*([^,]+)\s*,\s*([^)]+)\s*\)",
    r"(1.0*float(\2 <= \1 <= \3))",
    text
)

all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                  country_trans    =  BOL_trans,
                    make_fitted = True,        # make equations for fitted values of stochastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True     # Better for jupyter book 
                           ) 


Reading c:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln.wf1
Assummed model name: BOL
The model: BOL is unlinked 
Writing C:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln_modelflow.wf2
Model name: BOL

Processing the model:BOL
Check for Eviews @ which are not caught in the translation
Default WB var_group loaded
Variable description in wf1 file read
Default WB var_description loaded self.cty='' len(var_description)=16
var_description loaded from WF len(this)=301
testmodel calculated  
Calculation of add factors for BOL calculated  
